# Very basic `qsub`, `qstat`, `qdel` Python wrapper

## Basic code

In [110]:
# qsub_utils

import os
import subprocess
import platform


PBS_CONFIGURATION=""

def setup_configuration(nodes='', stdout='', stderr=''):
    """
    Method to setup PBS configuration
    :param nodes: specify nodes with list of features. `-l nodes={nodes}`
    :param stdout: specify stdout file. `-o {stdout}`
    :param stderr: specify stderr file. `-e {stderr}`
    """
    global PBS_CONFIGURATION
    conf = "#PBS"
    if len(nodes) > 0:
        conf += " -l nodes=%s" % nodes        
    if len(stdout) > 0:
        conf += " -o %s" % stdout
    if len(stderr) > 0:
        conf += " -e %s" % stderr
        
    PBS_CONFIGURATION=conf

    
def write_launch_file(cmd_str, name, cwd='', env=''):    
    """
    Method to write a PBS launch file for qsub
    
    :param cmd_str: list of 
    """
    assert len(cmd_str) > 0, "Job command can't be empty"
    assert len(name) > 0, "Job name can't be empty"
    filename = os.path.join(cwd, "job_%s.launch" % name)
    with open(filename, 'w') as w:
        conf = PBS_CONFIGURATION
        conf += " -N %s" % name
        if len(cwd) > 0:
            conf += " -d %s" % cwd        
        w.write(conf + '\n')        
        if len(env) > 0:
            w.write(env + '\n')
        w.write(cmd_str)    
        
    return filename
    
    
def submit_job(cmd, name, cwd='', env=''):
    """
    Method to submit a job writing a launch file and using qsub
    `qsub job_{name}.launch`
    
    :param cmd:
    """
                    
    filename = write_launch_file(' '.join(cmd), name, cwd, env)    
    program = ['qsub', filename]
    process = subprocess.Popen(program, 
                               stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT,
                               close_fds=False if platform.system() == 'Windows' else True)
    process.wait()    
    job_id = process.stdout.read()
    assert job_id is not None and len(job_id) > 0, "Failed to fetch job id from qsub"        
    return job_id, filename   


def delete_job(job_id):
    pass


def get_stats(job_id):
    _id = job_id.split('.')[0]
    
    program = ['qstat', '-f', _id]
    process = subprocess.Popen(program, 
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               close_fds=False if platform.system() == 'Windows' else True)
    process.wait() 
    out = process.stdout.read()
    err = process.stderr.read()    
    out = out.replace(r'(\t|\s)', '').split('\n')
    print "out split", out
    out
    stats = {}
    for line in out:
        kv = line.split(' = ')
        if len(kv) > 1:
            stats[kv[0]] = kv[1]
    print stats
    
    print "OUT: ", out
    print "ERR: ", err

    
def job_is_running(job_id):
    return False
    

In [ ]:
a = '  \ta = vb, \t b = ccc '
a.replace(r'(\t|\s)')

## Example

In [111]:
setup_configuration(nodes='1:knl7210:ram96gb', stdout='/home/u2459', stderr='/home/u2459')

In [107]:
caffe_help_cmd = [
    "caffe",
    "--help", 
]
job_id, filename = submit_job(caffe_help_cmd, 'caffe_help', env='export PATH=$PATH:/opt/caffe-master/build/tools/')

# while job_is_running(job_id):
#     pass


In [108]:
PBS_CONFIGURATION

'#PBS -l nodes=1:knl7210:ram96gb -o /home/u2459 -e /home/u2459'

In [112]:
python_inf_cmd = [
    "python",
    "-c \"import time;time.sleep(1000000000)\"", 
]
job_id, filename = submit_job(python_inf_cmd, 'python_inf', env='export PATH=$PATH:/opt/caffe-master/build/tools/')

# while job_is_running(job_id):
#     pass

get_stats(job_id)

out split ['Job Id: 3820.c001', '    Job_Name = python_inf', '    Job_Owner = u2459@c001', '    job_state = Q', '    queue = batch', '    server = c001', '    Checkpoint = u', '    ctime = Mon Mar 20 18:52:10 2017', '    Error_Path = c001:/home/u2459/python_inf.e3820', '    Hold_Types = n', '    Join_Path = n', '    Keep_Files = n', '    Mail_Points = n', '    mtime = Mon Mar 20 18:52:10 2017', '    Output_Path = c001:/home/u2459/python_inf.o3820', '    Priority = 0', '    qtime = Mon Mar 20 18:52:10 2017', '    Rerunable = True', '    Resource_List.nodect = 1', '    Resource_List.nodes = 1:knl7210:ram96gb', '    Resource_List.walltime = 24:00:00', '    Variable_List = PBS_O_QUEUE=batch,PBS_O_HOME=/home/u2459,', 'PBS_O_LOGNAME=u2459,', 'PBS_O_PATH=/opt/intel/intelpython27/bin/:/opt/intel/intelpython35/bin', '/:/opt/intel/compilers_and_libraries_2017.0.098/linux/mpi/intel64/bin:', '/opt/intel/compilers_and_libraries_2017.0.098/linux/bin/intel64:/opt/i', 'ntel/compilers_and_libraries_201

In [100]:
!cat {filename}

#PBS -l nodes=1:knl7210:ram96gb -o /home/u2459 -e /home/u2459 -N python_inf
export PATH=$PATH:/opt/caffe-master/build/tools/
python -c "import time;time.sleep(1000000000)"

In [95]:
!rm ~/python_inf.*

In [96]:
!ls ~/

build_opencv	     Intel_MobileODT  start_digits.sh	test_launch
DIGITS		     keras_source     start_digits.sh~	test.py
digits_dependencies  opencv	      STDIN.e3535	tmp
env.local	     opencv_source    STDIN.o3535


In [97]:
!qstat -f {job_id.split(".")[0]}

Job Id: 3815.c001
    Job_Name = python_inf
    Job_Owner = u2459@c001
    job_state = R
    queue = batch
    server = c001
    Checkpoint = u
    ctime = Mon Mar 20 18:47:53 2017
    Error_Path = c001:/home/u2459/python_inf.e3815
    exec_host = c001-n029/0
    Hold_Types = n
    Join_Path = n
    Keep_Files = n
    Mail_Points = n
    mtime = Mon Mar 20 18:47:54 2017
    Output_Path = c001:/home/u2459/python_inf.o3815
    Priority = 0
    qtime = Mon Mar 20 18:47:53 2017
    Rerunable = True
    Resource_List.nodect = 1
    Resource_List.nodes = 1:knl7210:ram96gb
    Resource_List.walltime = 24:00:00
    session_id = 235977
    Variable_List = PBS_O_QUEUE=batch,PBS_O_HOME=/home/u2459,
	PBS_O_LOGNAME=u2459,
	PBS_O_PATH=/opt/intel/intelpython27/bin/:/opt/intel/intelpython35/bin
	/:/opt/intel/compilers_and_libraries_2017.0.098/linux/mpi/intel64/bin:
	/opt/intel/compilers_and_libraries_2017.0.098/linux/bin/intel64:/opt/i
	ntel/compilers_and_libraries_2017.0.098/linux/mpi/intel64/bin:/op

In [98]:
get_stats(job_id)

out split ['']


ValueError: need more than 1 value to unpack

In [89]:
!qstat